# Lecture 1: Introduction to LLMs
**Source: https://github.com/IvanReznikov/DataVerse/blob/main/Courses/LangChain/Lecture1.%20Introduction%20to%20LangChain**

## Notes

[Langchain](https://docs.langchain.com/docs/) is a framework for developing applications that run off LLMs. 

Langchain supports a variety of models for example:
- Language models that can be used for text generation
- Text embedding models that are used to generate embeddings to be passed into other models. 

**Prompts** are the instructions we pass to LLMs. Prompts can take the form of strings or lists of messages. 

**Memory** is the component of Langchain that enables the retention and utilization of past interactions with the LLM. By default Langchain components are stateless and treat each query as independent. 

**Indexes** are the databases of information that provide context to the LLM. An index is usually stored in a vector database. 

**Chains** are sequences of calls to a LLM. This allows us to decompose complex tasks into simpler chains, that are linked together in sequence to get the results we want. 

**Agents** are objects that handle interaction between the user and the LLM. Tasks that the agent handles are like parsing input, generating output and determining the sequence of actions to follow and tools to use. 

**Tools** are functions designed to perform a specific task. Examples of tools are Google Search, Database lookups, API calls and other chains. The standard inference proceedure is to accept a string as input and returns a string as output. 


In [5]:
# Package imports
import requests 
import bs4
import pandas as pd

In [2]:
# UDFs
import requests
from bs4 import BeautifulSoup
import pandas as pd
def get_request(url, cookies={}, headers={}):
    return requests.get(url, cookies=cookies, headers=headers)

def collect_data(url):
    response = get_request(url)
    soup = BeautifulSoup(response.text, "lxml")
    table = soup.find("table", class_="publicholidays")
    return table

def convert_html_table_to_df(html_text):
    return pd.read_html(str(html_text))[0]

## Define the root path for the xml table
ROOT_URL = "https://publicholidays.ae/2023-dates/"

# Scrape data and convert to dataframe
html_text = collect_data(ROOT_URL)
df = convert_html_table_to_df(html_text)

/var/folders/fw/638xqct90tq7pjj8c9fsdj5r0000gn/T/ipykernel_10309/268516617.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(html_text))[0]


In [3]:
df

,Date,Day,Holiday
0,1 Jan,Sun,New Year's Day
1,20 Apr,Thu,Eid al-Fitr Holiday
2,21 Apr,Fri,Eid al-Fitr
3,22 Apr,Sat,Eid al-Fitr Holiday
4,23 Apr,Sun,Eid al-Fitr Holiday
5,27 Jun,Tue,Arafat Day
6,28 Jun,Wed,Eid al-Adha
7,29 Jun,Thu,Eid al-Adha Holiday
8,30 Jun,Fri,Eid al-Adha Holiday
9,21 Jul,Fri,Islamic New Year


In [9]:
df.iloc[:-1,:].to_csv("uae_holidays.csv")

In [20]:
# Langchain imports
from langchain.chat_models import ChatOpenAI # model
from langchain.llms import LlamaCpp
from langchain.indexes import VectorstoreIndexCreator # Index
from langchain.document_loaders.csv_loader import CSVLoader # loader
from langchain.prompts import PromptTemplate # prompt
from langchain.memory import ConversationBufferMemory # memory
from langchain.chains import RetrievalQA # Chain
import os


from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [16]:
MODEL_PATH = "/Users/dloader/Documents/LLM-models/TheBloke/dolphin-2.1-mistral-7B-GGUF/dolphin-2.1-mistral-7b.Q4_K_M.gguf"

def load_llm(MODEL_PATH):
    llm = LlamaCpp(
        streaming=True, 
        model_path=MODEL_PATH,
        temperature=0.7,
        top_p=1,
        n_ctx=4096,
        verbose=True, 
)
    
def load_index():
    loader = CSVLoader(file_path="uae_holidays.csv")
    index = VectorstoreIndexCreator().from_loaders([loader])
    return index

In [17]:
prompt_template = """
You are a assistant to help answer when are the official UAE holidays, based only on the data provided.
Context: {context}
-----------------------
History: {chat_history}
=======================
Human: {question}
Chatbot:
"""

# Create a prompt from the template
prompt = PromptTemplate(
    input_variables=["context", "chat_history", "question"], template=prompt_template
)

In [18]:
# Creating conversation memory
memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True, input_key="question"
)

In [23]:
!pwd

/Users/dloader/Documents/GitHub/tutorial-hell/DataVerse/Lecture-1-Introduction-To-Langchain


In [24]:
# Handle vector store instead of using the OPENAPI
# Setup the sentence transformer emb
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# Create an embedding store
vector_store = FAISS.from_documents("uae_holidays.csv", embedding=embeddings)
vector_store

AttributeError: 'str' object has no attribute 'page_content'

In [21]:
# Create QA 
qa = RetrievalQA.from_chain_type(
    llm=load_llm(MODEL_PATH), 
    chain_type="stuff", 
    retreiver = load_index().vectorstore.as_retriever(),
    verbose=True, 
    chain_type_kwargs={
        "prompt": prompt,
        "memory": memory,
    }
    
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/dloader/Documents/LLM-models/TheBloke/dolphin-2.1-mistral-7B-GGUF/dolphin-2.1-mistral-7b.Q4_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32002,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q4_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tenso

ValidationError: 1 validation error for LLMChain
llm
  none is not an allowed value (type=type_error.none.not_allowed)